# OpenAI Swarm

## Use case 1

In [1]:
import os
from openai import AzureOpenAI
from duckduckgo_search import DDGS
from swarm import Swarm, Agent
from datetime import datetime

clientOI = AzureOpenAI(
    api_key="abc",  
    api_version="2023-03-15-preview",
    azure_endpoint = "https://ai-proxy.lab.samy.com"
    )

# Initialize Swarm client
client = Swarm(client=clientOI)
current_date = datetime.now().strftime("%Y-%m")

In [2]:
# 1. Create Internet Search Tool

def get_news_articles(topic):
    print(f"Running DuckDuckGo news search for {topic}...")
    
    # DuckDuckGo search
    ddg_api = DDGS()
    results = ddg_api.text(f"{topic} {current_date}", max_results=5)
    if results:
        news_results = "\n\n".join([f"Title: {result['title']}\nURL: {result['href']}\nDescription: {result['body']}" for result in results])
        return news_results
    else:
        return f"Could not find news results for {topic}."

In [3]:
# 2. Create AI Agents

# News Agent to fetch news
news_agent = Agent(
    name="News Assistant",
    instructions="You provide the latest news articles for a given topic using DuckDuckGo search.",
    functions=[get_news_articles],
    model="gpt-4o"
)

# Editor Agent to edit news
editor_agent = Agent(
    name="Editor Assistant",
    instructions="Rewrite and give me as news article ready for publishing. Each News story in separate section.",
    model="gpt-4o"
)


In [4]:
# 3. Create workflow

def run_news_workflow(topic):
    print("Running news Agent workflow...")
    
    # Step 1: Fetch news
    news_response = client.run(
        agent=news_agent,
        messages=[{"role": "user", "content": f"Get me the news about {topic} on {current_date}"}],
    )
    
    raw_news = news_response.messages[-1]["content"]
    
    # Step 2: Pass news to editor for final review
    edited_news_response = client.run(
        agent=editor_agent,
        messages=[{"role": "user", "content": raw_news }],
    )
    
    return edited_news_response.messages[-1]["content"]

In [5]:
# Example of running the news workflow for a given topic
print(run_news_workflow("AI"))

Running news Agent workflow...
Running DuckDuckGo news search for AI...
## Donald Trump's Election Victory Poised to Overturn Biden AI Policies

In a historic turn of events, Donald Trump has emerged victorious in the 2024 U.S. presidential election. His win sets the stage for a major overhaul of current federal policies on artificial intelligence (AI). As he prepares to take office, all eyes are on the impending changes to the AI regulations implemented during Joe Biden's administration. This development is projected to impact various sectors reliant on AI technologies. [Read more](https://arstechnica.com/ai/2024/11/trump-victory-signals-major-shakeup-for-us-ai-regulations/)

## Biden Administration Releases Comprehensive AI National Security Memorandum

On October 24, 2024, the Biden administration unveiled a pivotal document detailing strategies for AI within the sphere of national security. This memorandum outlines a timeline and an index of responsibilities intended to fortify the

## Use case 2

In [6]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from duckduckgo_search import DDGS
from swarm import Swarm, Agent
from datetime import datetime
from sentence_transformers import util
from sentence_transformers import SentenceTransformer

# Set up AzureOpenAI client
clientOI = AzureOpenAI(
    api_key="abc",
    api_version="2023-03-15-preview",
    azure_endpoint="https://ai-proxy.lab.samy.com"
)

# Initialize Swarm client
client = Swarm(client=clientOI)
current_date = datetime.now().strftime("%Y-%m")

# Load the mpnet model from sentence-transformers
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [7]:
# Sample RAG DataFrame
def create_rag_dataframe():
    data = {
        "text": [
            "Machine learning is a subset of artificial intelligence.",
            "Python is commonly used for data science and machine learning.",
            "Deep learning algorithms require large amounts of data.",
            "Transformers are state-of-the-art models for NLP tasks."
        ],
        "topic": ["AI", "Python", "AI", "NLP"]
    }
    
    # Generate mpnet embeddings for each text entry
    data['embedding'] = [model.encode(text) for text in data['text']]
    
    # Convert to DataFrame
    df = pd.DataFrame(data)
    return df

# Create the RAG DataFrame
rag_df = create_rag_dataframe()
rag_df

C:\Users\Abhishek_Jaiswal\AppData\Local\miniconda3\envs\mlenv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


C:\Users\Abhishek_Jaiswal\AppData\Local\miniconda3\envs\mlenv\Lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\Abhishek_Jaiswal\AppData\Local\miniconda3\envs\mlenv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Abhishek_Jaiswal\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

C:\Users\Abhishek_Jaiswal\AppData\Local\miniconda3\envs\mlenv\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

,text,topic,embedding
0,Machine learning is a subset of artificial int...,AI,"[-0.02844554, -0.0067215366, -0.07253845, -0.0..."
1,Python is commonly used for data science and m...,Python,"[0.012467294, 0.02821976, -0.035361845, -0.021..."
2,Deep learning algorithms require large amounts...,AI,"[-0.029015891, 0.122301586, -0.030206438, 0.03..."
3,Transformers are state-of-the-art models for N...,NLP,"[0.06630315, 0.05206603, -0.023457907, 0.02128..."


In [8]:
# 1. Internet Search Tool
def get_news_articles(topic):
    print(f"Running DuckDuckGo news search for {topic}...")
    ddg_api = DDGS()
    results = ddg_api.text(f"{topic} {current_date}", max_results=5)
    if results:
        news_results = "\n\n".join(
            [f"Title: {result['title']}\nURL: {result['href']}\nDescription: {result['body']}" for result in results]
        )
        return news_results
    else:
        return f"Could not find news results for {topic}."

# RAG Retrieval Function
def retrieve_rag_answer(topic):
    # Generate the mpnet embedding for the input topic
    topic_embedding = model.encode(topic)

    # Compute cosine similarity between topic_embedding and each embedding in rag_df
    rag_df['similarity'] = rag_df['embedding'].apply(lambda x: util.cos_sim(x, topic_embedding).item())

    # Retrieve the most similar row
    relevant_row = rag_df.loc[rag_df['similarity'].idxmax()]

    # Return the text if similarity is above a threshold, otherwise return a default response
    return relevant_row['text'] if relevant_row['similarity'] > 0.5 else "No relevant information found."


# 2. Create AI Agents
# Greeting Agent
greeting_agent = Agent(
    name="Greeting Agent",
    instructions="Respond with a friendly greeting.",
    model="gpt-4o"
)

# Irrelevant Agent
irrelevant_agent = Agent(
    name="Irrelevant Agent",
    instructions="Handle irrelevant queries by politely asking the user to rephrase.",
    model="gpt-4o"
)

# News Agent for online search
news_agent = Agent(
    name="News Agent",
    instructions="Provide the latest news articles for a given topic using DuckDuckGo search.",
    functions=[get_news_articles],
    model="gpt-4o"
)

# RAG Agent
rag_agent = Agent(
    name="RAG Agent",
    instructions="Retrieve answers from a knowledge base for queries on specific topics.",
    functions=[retrieve_rag_answer],
    model="gpt-4o"
)

# 3. Define bot workflow
def conversation_bot(user_input):
    if any(greet in user_input.lower() for greet in ["hello", "hi", "hey"]):
        response = client.run(agent=greeting_agent, messages=[{"role": "user", "content": user_input}])
        return response.messages[-1]["content"]

    elif "news" in user_input.lower():
        topic = user_input.split("about")[-1].strip()
        response = client.run(agent=news_agent, messages=[{"role": "user", "content": f"Get news on {topic}"}])
        return response.messages[-1]["content"]
    
    elif any(topic in user_input.lower() for topic in rag_df['topic'].unique()):
        topic = [topic for topic in rag_df['topic'].unique() if topic in user_input.lower()][0]
        response = client.run(agent=rag_agent, messages=[{"role": "user", "content": f"Retrieve info on {topic}"}])
        return response.messages[-1]["content"]
    
    else:
        response = client.run(agent=irrelevant_agent, messages=[{"role": "user", "content": user_input}])
        return response.messages[-1]["content"]

In [9]:
# Example usage
print(conversation_bot("Hello!"))

Hello there! How can I assist you today?


In [12]:
print(conversation_bot("Get me news about AI"))

Running DuckDuckGo news search for AI...
Here are the latest news articles on AI:

1. **Trump plans to dismantle Biden AI safeguards after victory**  
   Early Wednesday morning, Donald Trump became the presumptive winner of the 2024 US presidential election, setting the stage for dramatic changes to federal AI policy when he takes office.  
   [Read more here](https://arstechnica.com/ai/2024/11/trump-victory-signals-major-shakeup-for-us-ai-regulations/)

2. **The 2024 National Security Memorandum on AI: A Timeline and Index of Responsibilities**  
   On October 24, 2024, the Biden administration published a memorandum focused on advancing the United States' leadership in AI, harnessing AI for national security, and fostering safety and trustworthiness.  
   [Read more here](https://georgetownsecuritystudiesreview.org/2024/11/04/the-2024-national-security-memorandum-on-ai-a-timeline-and-index-of-responsibilities/)

3. **Fact Sheet: Key AI Accomplishments in the Year Since the Biden Adm

In [10]:
print(conversation_bot("Tell me something about Python"))

Python is a versatile, high-level programming language known for its readability and simplicity. Created by Guido van Rossum and first released in 1991, Python supports multiple programming paradigms, including procedural, object-oriented, and functional programming. Its clear syntax allows developers to express concepts in fewer lines of code compared to other languages, making it a great choice for both beginners and experienced programmers.

One of Python's key strengths is its extensive standard library and a vibrant ecosystem of third-party packages, which extend its capabilities in areas such as web development, data analysis, artificial intelligence, scientific computing, and more. The Python Package Index (PyPI) hosts thousands of modules that can be easily installed to enhance functionality.

Python is known for its large and active community, providing plenty of resources, tutorials, and forums for support. This popularity has led to its use in a wide range of applications, f

In [11]:
print(conversation_bot("What is the weather today?"))

I'm sorry, I can't provide real-time weather updates. You might want to check a weather app or website for the latest information.


# Use case 3

In [21]:
import os
from openai import AzureOpenAI
from swarm import Swarm, Agent
from datetime import datetime

# Initialize Azure OpenAI client
clientOI = AzureOpenAI(
    api_key="abc",
    api_version="2023-03-15-preview",
    azure_endpoint="https://ai-proxy.lab.samy.com"
)

# Initialize Swarm client
client = Swarm(client=clientOI)
current_date = datetime.now().strftime("%Y-%m")

# 1. Define Compliance Impact Assessment Agent

def assess_regulation_impact(regulation_text):
    """
    Analyzes the impact of a new regulation on existing documents.
    """
    # Placeholder function for analysis
    print(f"Analyzing regulation impact for: {regulation_text}")
    # Simulated output
    impact_results = f"Identified changes required for compliance with regulation: '{regulation_text}'."
    return impact_results

# 2. Define Amendment Suggestion Agent

def suggest_amendments(document_text):
    """
    Suggests amendments for the given document text based on compliance requirements.
    """
    # Placeholder function for suggesting amendments
    print(f"Suggesting amendments for document: {document_text}")
    # Simulated output
    amendment_suggestions = f"Suggested amendments for document: '{document_text}'."
    return amendment_suggestions

# 3. Create AI Agents

# Agent to assess compliance impact
compliance_agent = Agent(
    name="Compliance Assistant",
    instructions="Analyze new regulations and identify required changes in existing documents.",
    functions=[assess_regulation_impact],
    model="gpt-4o"
)

# Agent to suggest amendments
amendment_agent = Agent(
    name="Amendment Assistant",
    instructions="Provide suggested amendments for documents based on compliance requirements.",
    functions=[suggest_amendments],
    model="gpt-4o"
)

# 4. Define Workflow

def run_compliance_workflow(regulation_text, document_text):
    print("Running compliance workflow...")

    # Step 1: Assess regulatory impact
    compliance_response = client.run(
        agent=compliance_agent,
        messages=[{"role": "user", "content": f"Assess impact of regulation: {regulation_text}"}],
    )

    impact_analysis = compliance_response.messages[-1]["content"]

    # Step 2: Suggest amendments for affected document
    amendment_response = client.run(
        agent=amendment_agent,
        messages=[{"role": "user", "content": f"Suggest amendments for document: {document_text}"}],
    )

    amendments = amendment_response.messages[-1]["content"]

    # Combine the results
    return f"Impact Analysis:\n{impact_analysis}\n\nSuggested Amendments:\n{amendments}"

# Example of running the compliance workflow with a sample regulation and document
regulation_text = "New financial reporting requirements for Q4 2024."
document_text = "Existing contract outlining financial obligations."

print(run_compliance_workflow(regulation_text, document_text))

Running compliance workflow...
Analyzing regulation impact for: New financial reporting requirements for Q4 2024
Suggesting amendments for document: Existing contract outlining financial obligations.
Suggesting amendments for document: Existing contract outlining financial obligations.
Suggesting amendments for document: Existing contract outlining financial obligations.
Impact Analysis:
The assessment of the new financial reporting requirements for Q4 2024 has identified changes necessary for compliance. If you need further details on the specific documents affected or additional information on how to implement these changes, please let me know!

Suggested Amendments:
I am currently unable to process the document for compliance suggestions. Could you please provide more specific details or the content of the document you would like amended?


In [ ]:
### Problem:
#Sir needs to assess the impact of regulatory or business changes on existing contracts, identify which documents require amendments, and suggest specific changes. The current process is manual, time-consuming, and lacks clarity.

### Solution:
#We created a workflow using AI agents:
#1. **Compliance Agent**: Analyzes the impact of new regulations on existing documents.
#2. **Amendment Agent**: Suggests amendments for the affected documents based on the compliance requirements.
#This automated workflow significantly reduces manual effort and provides clear, actionable insights, making compliance management more efficient and streamlined.

# Use case 4

In [28]:
import os
from openai import AzureOpenAI
from datetime import datetime
from swarm import Swarm, Agent

# Mocking TaskManager class
class TaskManager:
    def get_tasks(self, persona):
        # Simulate fetching tasks based on persona
        tasks = {
            "General Counsel": ["Review contracts", "Attend leadership meeting", "Update matter status"],
            "Legal Ops": ["Monitor contract turnaround times", "Review legal KPIs"],
            "Lawyer Manager": ["Assign tasks to team", "Review case progress"],
            "Lawyer": ["Prepare legal documents", "Meet with clients", "Submit case reports"]
        }
        return tasks.get(persona, [])

# Mocking DocumentParser class
class DocumentParser:
    def extract_dates_and_tasks(self, document_path):
        # Simulate extracting dates and tasks from documents
        return [
            {"date": "2024-11-07", "event": "Contract Review Deadline"},
            {"date": "2024-11-10", "event": "Client Meeting"}
        ]

# AzureOpenAI client setup
clientOI = AzureOpenAI(
    api_key="abc",
    api_version="2023-03-15-preview",
    azure_endpoint="https://ai-proxy.lab.samy.com"
)

# Initialize Swarm client
client = Swarm(client=clientOI)
current_date = datetime.now().strftime("%Y-%m")

# Initialize TaskManager and DocumentParser
task_manager = TaskManager()  # Using the mock TaskManager class
document_parser = DocumentParser()  # Using the mock DocumentParser class

# 1. Integrate AI to create Persona-specific Dashboards
def generate_dashboard(persona):
    print(f"Generating {persona} dashboard...")

    if persona == "General Counsel":
        dashboard = get_gc_dashboard()
    elif persona == "Legal Ops":
        dashboard = get_legal_ops_dashboard()
    elif persona == "Lawyer Manager":
        dashboard = get_lawyer_manager_dashboard()
    elif persona == "Lawyer":
        dashboard = get_lawyer_dashboard()

    return dashboard

# Define Dashboard Generation Functions (using LLM to fetch data)
def get_gc_dashboard():
    # Collect and aggregate data for GC
    matter_data = get_matter_data("GC")
    budget_data = get_budget_data("GC")
    
    # LLM for generating a holistic view
    dashboard = f"General Counsel Dashboard: {matter_data}\nBudget Overview: {budget_data}"
    return dashboard

def get_legal_ops_dashboard():
    # Collect KPIs and performance data
    kpi_data = get_kpi_data("Legal Ops")
    return f"Legal Ops Dashboard: {kpi_data}"

def get_lawyer_manager_dashboard():
    # Get matters and team performance
    team_matter_data = get_team_matter_data("Lawyer Manager")
    return f"Lawyer Manager Dashboard: {team_matter_data}"

def get_lawyer_dashboard():
    # Get individual matters and tasks
    personal_matter_data = get_personal_matter_data("Lawyer")
    return f"Lawyer Dashboard: {personal_matter_data}"

# Missing function definitions
def get_matter_data(persona):
    # Simulate fetching matter data for a specific persona
    return f"Matter Data for {persona}: [Matter 1, Matter 2, Matter 3]"

def get_budget_data(persona):
    # Simulate fetching budget data for a specific persona
    return f"Budget Data for {persona}: [Budget 1, Budget 2, Budget 3]"

def get_kpi_data(persona):
    # Simulate fetching KPI data for a specific persona
    return f"KPI Data for {persona}: [KPI 1, KPI 2, KPI 3]"

def get_team_matter_data(persona):
    # Simulate fetching team matter data for a specific persona
    return f"Team Matter Data for {persona}: [Team Matter 1, Team Matter 2]"

def get_personal_matter_data(persona):
    # Simulate fetching personal matter data for a specific persona
    return f"Personal Matter Data for {persona}: [Personal Matter 1, Personal Matter 2]"

# 2. Automated Task Management (sync with task manager)
def sync_with_task_manager(persona):
    tasks = task_manager.get_tasks(persona)
    return tasks

# 3. Document Parsing (generate chronology from documents)
def generate_chronology_from_documents():
    document_data = document_parser.extract_dates_and_tasks("matter_documents_path")
    return document_data

# 4. Create AI Agents for Persona-Based Workflows
dashboard_agent = Agent(
    name="Dashboard Assistant",
    instructions="Provide personalized dashboard data for the assigned persona.",
    model="gpt-4o",
)

# 5. Workflow: Combine all into a unified workflow
def run_persona_workflow(persona):
    print("Running persona-based workflow...")

    # Step 1: Generate Persona Dashboard
    dashboard_data = generate_dashboard(persona)
    
    # Step 2: Sync with Task Manager for updates
    task_data = sync_with_task_manager(persona)
    
    # Step 3: Parse documents for key dates and events
    chronology_data = generate_chronology_from_documents()
    
    # Step 4: Combine all the data for a comprehensive view
    full_dashboard = f"{dashboard_data}\n{task_data}\nChronology of Events: {chronology_data}"
    
    return full_dashboard


In [29]:
# Example usage for the "General Counsel" persona
print(run_persona_workflow("General Counsel"))

Running persona-based workflow...
Generating General Counsel dashboard...
General Counsel Dashboard: Matter Data for GC: [Matter 1, Matter 2, Matter 3]
Budget Overview: Budget Data for GC: [Budget 1, Budget 2, Budget 3]
['Review contracts', 'Attend leadership meeting', 'Update matter status']
Chronology of Events: [{'date': '2024-11-07', 'event': 'Contract Review Deadline'}, {'date': '2024-11-10', 'event': 'Client Meeting'}]


In [30]:
### Problem:
#Legal departments often involve multiple personas such as **General Counsel (GC)**, **Legal Operations (Legal Ops)**, **Lawyer Managers**, and **Lawyers**, each requiring personalized access to specific legal data. Gathering and presenting this information manually from various sources is time-consuming and error-prone. These personas need insights into ongoing legal matters, budgets, KPIs, tasks, and document-based events, which are often scattered across different systems.

### Solution:
#The solution automates the process of generating tailored dashboards for each persona using **AI agents** and task management integration. The core components include:

#1. **Persona-based Dashboards**: The code generates dashboards for each persona (e.g., GC, Legal Ops) by aggregating relevant data such as matter details, budgets, and tasks using LLM models.
#2. **Task Management**: It integrates with a task management system to fetch real-time tasks for each persona.
#3. **Document Parsing**: It uses a **DocumentParser** to extract key dates and milestones from legal documents, creating a chronology for each matter.
#4. **AI Agents**: Personalized dashboards are created by running workflows that leverage AI models for data aggregation and timeline generation.

#By automating this process, the solution streamlines data access and improves decision-making, providing legal teams with a consolidated view of all necessary information, thus increasing productivity and accuracy.

# Use case 5

In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from duckduckgo_search import DDGS
from swarm import Swarm, Agent
from datetime import datetime
from sentence_transformers import util
from sentence_transformers import SentenceTransformer

# Set up AzureOpenAI client
clientOI = AzureOpenAI(
    api_key="abc",
    api_version="2023-03-15-preview",
    azure_endpoint="https://ai-proxy.lab.samy.com"
)

# Initialize Swarm client
client = Swarm(client=clientOI)
current_date = datetime.now().strftime("%Y-%m")

# Load the mpnet model from sentence-transformers
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Sample RAG DataFrame
def create_rag_dataframe():
    data = {
        "text": [
            "Machine learning is a subset of artificial intelligence.",
            "Python is commonly used for data science and machine learning.",
            "Deep learning algorithms require large amounts of data.",
            "Transformers are state-of-the-art models for NLP tasks."
        ],
        "topic": ["AI", "Python", "AI", "NLP"]
    }
    
    # Generate mpnet embeddings for each text entry
    data['embedding'] = [model.encode(text) for text in data['text']]
    
    # Convert to DataFrame
    df = pd.DataFrame(data)
    return df

# Create the RAG DataFrame
rag_df = create_rag_dataframe()

# 1. Internet Search Tool
def get_news_articles(topic):
    print(f"Running DuckDuckGo news search for {topic}...")
    ddg_api = DDGS()
    results = ddg_api.text(f"{topic} {current_date}", max_results=5)
    if results:
        news_results = "\n\n".join(
            [f"Title: {result['title']}\nURL: {result['href']}\nDescription: {result['body']}" for result in results]
        )
        return news_results
    else:
        return f"Could not find news results for {topic}."

# RAG Retrieval Function
def retrieve_rag_answer(topic):
    topic_embedding = model.encode(topic)
    rag_df['similarity'] = rag_df['embedding'].apply(lambda x: util.cos_sim(x, topic_embedding).item())
    relevant_row = rag_df.loc[rag_df['similarity'].idxmax()]
    return relevant_row['text'] if relevant_row['similarity'] > 0.5 else "No relevant information found."

# 2. Create AI Agents
greeting_agent = Agent(name="Greeting Agent", instructions="Respond with a friendly greeting.", model="gpt-4o")
irrelevant_agent = Agent(name="Irrelevant Agent", instructions="Handle irrelevant queries by politely asking the user to rephrase.", model="gpt-4o")
news_agent = Agent(name="News Agent", instructions="Provide the latest news articles for a given topic using DuckDuckGo search.", functions=[get_news_articles], model="gpt-4o")
rag_agent = Agent(name="RAG Agent", instructions="Retrieve answers from a knowledge base for queries on specific topics.", functions=[retrieve_rag_answer], model="gpt-4o")

# Classification agent for query intent
classifier_agent = Agent(
    name="Classifier Agent",
    instructions="Classify the user query into one of the following types: greeting, news, rag, or irrelevant.",
    model="gpt-4o"
)

# Updated classify_user_query function using the classifier agent
def classify_user_query(user_input):
    classification_response = client.run(agent=classifier_agent, messages=[{"role": "user", "content": user_input}])
    query_type = classification_response.messages[-1]["content"].strip().lower()
    return query_type

# Updated conversation bot workflow
def conversation_bot(user_input):
    query_type = classify_user_query(user_input)
    
    if query_type == "greeting":
        response = client.run(agent=greeting_agent, messages=[{"role": "user", "content": user_input}])
    
    elif query_type == "news":
        topic = user_input.split("about")[-1].strip()
        response = client.run(agent=news_agent, messages=[{"role": "user", "content": f"Get news on {topic}"}])
    
    elif query_type == "rag":
        topic = [topic for topic in rag_df['topic'].unique() if topic.lower() in user_input.lower()][0]
        response = client.run(agent=rag_agent, messages=[{"role": "user", "content": f"Retrieve info on {topic}"}])
    
    else:
        response = client.run(agent=irrelevant_agent, messages=[{"role": "user", "content": user_input}])
    
    return response.messages[-1]["content"]

In [32]:
print(conversation_bot("Get me news about AI"))

Running DuckDuckGo news search for AI...
Here are some of the latest news articles on AI:

1. **[The 2024 National Security Memorandum on AI](https://georgetownsecuritystudiesreview.org/2024/11/04/the-2024-national-security-memorandum-on-ai-a-timeline-and-index-of-responsibilities/)**: An article exploring the Biden administration's efforts to advance the United States' leadership in AI while addressing national security and fostering AI safety. 

2. **[The 5 Biggest Artificial Intelligence Trends For 2024 - Forbes](https://www.forbes.com/sites/bernardmarr/2023/11/01/the-top-5-artificial-intelligence-trends-for-2024/)**: Covers new AI trends for 2024, including quantum advancements and generative AI, influencing human-AI collaborations.

3. **[Research: How Gen AI Is Already Impacting the Labor Market](https://hbr.org/2024/11/research-how-gen-ai-is-already-impacting-the-labor-market)**: Analyzes how general AI tools like ChatGPT and image-generating technologies are affecting online gi